In [1]:
import pandas as pd
import statsmodels.formula.api as sm
import numpy as np

In [2]:
df = pd.read_csv("bank.csv", sep = ";")

In [3]:
df['ny'] = df['y'].apply(lambda x: 1 if x == 'yes' else 0)

In [4]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,ny
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no,0
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no,0
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no,0
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no,0
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no,0


In [5]:
df.ny.unique()

array([0, 1])

In [6]:
df['job'] = df['job'].astype('category')
df['marital'] = df['marital'].astype('category')
df['education'] = df['education'].astype('category')

## Предскажите результаты маркетинговой кампании (подпишется ли клиент на продукт или нет)

## Модель 1 
#### Гипотезы:
1. **Образование** может быть связано с уровнем дохода, значит люди могут быть более склонны к подписке на продукт.
2. **Статус работы**: люди с более стабильной работой могут быть более склонны к подписке на продукт.
3. **Семейное положение**: люди, состоящие в браке или имеющие стабильные отношения, также могут быть более склонны к подписке на продукт.

In [7]:
model = sm.logit(formula="ny~ education + job + marital", data= df).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.347991
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                     ny   No. Observations:                 4521
Model:                          Logit   Df Residuals:                     4504
Method:                           MLE   Df Model:                           16
Date:                Thu, 26 Oct 2023   Pseudo R-squ.:                 0.02614
Time:                        11:47:35   Log-Likelihood:                -1573.3
converged:                       True   LL-Null:                       -1615.5
Covariance Type:            nonrobust   LLR p-value:                 2.585e-11
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1.8961      0.237     -8.004      0.000      -2.360      -1.432
education[T.secondary]     0.1208      0.164      0.738      0.460      -0.200       0.441
education[T.tertiary]      0.3696      0.188      1.964      0.050       0.001       0.739
education[T.unknown]      -0.1262      0.289     -0.436      0.663      -0.693       0.441
job[T.blue-collar]        -0.4295      0.196     -2.191      0.028      -0.814      -0.045
job[T.entrepreneur]       -0.3277      0.312     -1.051      0.293      -0.939       0.284
job[T.housemaid]           0.1327      0.329      0.404      0.686      -0.511       0.777
job[T.management]         -0.0485      0.195     -0.249      0.803      -0.430       0.333
job[T.retired]             0.9094      0.218      4.170      0.000       0.482       1.337
job[T.self-employed]      -0.1653      0.282     -0.585      0.558      -0.719       0.388
job[T.services]           -0.2971      0.221     -1.344      0.179      -0.730       0.136
job[T.student]             0.6181      0.306      2.023      0.043       0.019       1.217
job[T.technician]         -0.1873      0.184     -1.016      0.310      -0.549       0.174
job[T.unemployed]         -0.2160      0.327     -0.660      0.509      -0.857       0.425
job[T.unknown]             0.6466      0.455      1.421      0.155      -0.245       1.538
marital[T.married]        -0.3943      0.141     -2.798      0.005      -0.670      -0.118
marital[T.single]         -0.0485      0.155     -0.314      0.754      -0.352       0.255
==========================================================================================
"""

In [8]:
model_odds = pd.DataFrame(np.exp(model.params), columns= ['OR'])
model_odds['p-value']= model.pvalues
model_odds[['2.5%', '97.5%']] = np.exp(model.conf_int())
model_odds

,OR,p-value,2.5%,97.5%
Intercept,0.150148,1.200968e-15,0.094380,0.238868
education[T.secondary],1.128416,4.602691e-01,0.818840,1.555033
education[T.tertiary],1.447154,4.957358e-02,1.000689,2.092811
education[T.unknown],0.881427,6.627458e-01,0.499864,1.554250
job[T.blue-collar],0.650814,2.843323e-02,0.443204,0.955674
job[T.entrepreneur],0.720544,2.933388e-01,0.390998,1.327844
job[T.housemaid],1.141895,6.863387e-01,0.599708,2.174264
job[T.management],0.952664,8.032637e-01,0.650504,1.395177
job[T.retired],2.482884,3.043323e-05,1.619309,3.807002
job[T.self-employed],0.847671,5.584038e-01,0.487361,1.474362


### Выводы по модели 1: 
- Псевдо R-квадрат = 0.02614 указывает на низкую объясняющую способность модели.
- LL-Null = -1615.5, все таки модель с независимыми переменными лучше, чем без них. 

- Высшее образование имеет статистически значимое влияние на результат подписки на продукт.
- Среди типов занятости, наибольшее влияние оказывает статус "retired": люди, находящиеся на пенсии, имеют наибольшую вероятность положительного результата. У студентов верятность меньше, но тоже значимая (p = 0.043). А вот рабочий класс (blue-collar) значимо связан с зависимой переменной (р = 0.028), однако в сторону решения отсутствия подписки.  
- Семейное положение также имеет значимое влияние на результат: значение married имеет отрицательный коэффициент, что может означать, что люди, состоящие в браке, имеют меньшую вероятность подписки на продукт, тогда как стат. связи неженатых с зависимой переменной не обнаружено.



In [9]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y', 'ny'],
      dtype='object')

## Модель 2
#### Гипотезы:
1. Убираем **статус работы**, так как большая часть статустов не имеет связи с зависимой переменной.
2. Добавляю количественную переменную **Возраст**: Возраст может быть с финансовой стабильностью, что увеличит вероятность подписки на продукт.
2. **Баланс на счету**: люди с высоким балансом на счету могут быть более склонны к подписке на продукт, так как у них есть средства для его оплаты.

In [10]:
model= sm.logit(formula="ny~ education + age + balance + marital", data= df).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.351348
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                     ny   No. Observations:                 4521
Model:                          Logit   Df Residuals:                     4513
Method:                           MLE   Df Model:                            7
Date:                Thu, 26 Oct 2023   Pseudo R-squ.:                 0.01675
Time:                        11:47:36   Log-Likelihood:                -1588.4
converged:                       True   LL-Null:                       -1615.5
Covariance Type:            nonrobust   LLR p-value:                 2.232e-09
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -3.1379      0.305    -10.297      0.000      -3.735      -2.541
education[T.secondary]     0.2127      0.152      1.397      0.162      -0.086       0.511
education[T.tertiary]      0.5129      0.158      3.236      0.001       0.202       0.823
education[T.unknown]       0.0426      0.277      0.154      0.878      -0.500       0.585
marital[T.married]        -0.3816      0.140     -2.728      0.006      -0.656      -0.107
marital[T.single]          0.1745      0.161      1.086      0.277      -0.140       0.489
age                        0.0236      0.005      4.872      0.000       0.014       0.033
balance                  7.97e-06   1.44e-05      0.554      0.579   -2.02e-05    3.61e-05
==========================================================================================
"""

In [11]:
model_odds = pd.DataFrame(np.exp(model.params), columns= ['OR'])
model_odds['p-value']= model.pvalues
model_odds[['2.5%', '97.5%']] = np.exp(model.conf_int())
model_odds

,OR,p-value,2.5%,97.5%
Intercept,0.043376,7.261261e-25,0.023870,0.078820
education[T.secondary],1.236971,1.623936e-01,0.917882,1.666986
education[T.tertiary],1.670094,1.210656e-03,1.224185,2.278424
education[T.unknown],1.043529,8.777262e-01,0.606415,1.795722
marital[T.married],0.682781,6.368897e-03,0.519069,0.898125
marital[T.single],1.190651,2.772690e-01,0.869104,1.631161
age,1.023853,1.104429e-06,1.014189,1.033608
balance,1.000008,5.792572e-01,0.999980,1.000036


### Выводы по модели 2:
- Log-Likelihood уменьшился, по сравнению с прошлой моделью (-1573.3 vs -1588.4), в первом случае подгонка модели к данным лучше.
- LL-Null = -1615.5, все таки модель с независимыми переменными лучше, чем без них. 
- Псевдо R квадрат = 0.01675. Новая модель еще хуже объясняет результ подписки.

- Высшее образование имеет статистически значимое влияние на результат подписки на продукт.
- Состояние семейного положения также оказывает влияние на зависимую переменную. Значение married имеет отрицательный коэффициент, что может означать, что люди, состоящие в браке, имеют меньшую вероятность наступления исследуемого события.
- Переменная age имеет положительный коэффициент 0.0236 (р = 0.000), что указывает на то, что с увеличением возраста вероятность наступления исследуемого события возрастает.
- Переменная balance имеет коэффициент близкий к нулю (7.97e-06) и не является статистически значимой (P>|z| = 0.579). Это указывает на то, что баланс на счете клиента не имеет значительного влияния на зависимую переменную.

## Выводы по гипотезам 
##### Подтвердились следующие гипотезы:

1. **Образование**: образование имеет статистически значимое влияние на результат подписки на продукт.
2. **Возраст**: возраст действительно имеет положительное влияние на результат подписки на продукт.

##### Частично подтвердились:

1. **Семейное положение**: состояние семейного положения оказывает влияние на зависимую переменную, правда, оказалось, что в сторону отсутствия подписки.
2. **Статус работы**: некоторые неочевидные статусы работы (пенсионеры, студенты) имеют связь с положительным влиянием на результат подписки на продукт, тогда как менеджеры, ИП-шники, администраторы и т.д - нет.

##### Не подтвердились:

1. **Баланс на счету**: баланс на счету клиента не имеет значительного влияния на результат подписки на продукт.

## Таким образом, портрет потенциального подписчика по результатам моего моделирования выглядит следующим образом: 
Это пенсионер или студент, но точно не принадлежащий к рабочему классу. Желательно, чтобы он не состоял в браке. Хорошо бы, чтобы он имел высшее образование. Чем старше наш клиент, тем лучше! Баланс на его счете не имеет значения.